In [ ]:
import arcpy
import os
ws = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb'
arcpy.env.workspace = ws

In [ ]:
def unique_values(table, field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
    
        return sorted({row[0] for row in cursor if row[0] != None})

In [ ]:
def pubFind(searchTerms, myValues):
    pubLand = []
    myValues = [value.upper() for value in myValues]
    
    for owner in myValues:
        for term in searchTerms:
            if term in owner:
                pubLand.append(owner)
    return pubLand

In [ ]:
def removeTerms(OgList, removeList):
    for item in removeList:
        if item in OgList:
            OgList.remove(item)
    return OgList
    

In [ ]:
def removePart(OgList, strPrtList):
    newRemoveList = []
   
    for i in OgList:
        for string in strPrtList:
            if string in i:
                
                newRemoveList.append(i)
    
    
    for item in newRemoveList:
       
        if item in OgList:
            OgList.remove(item)
    
    return OgList, newRemoveList
    


In [ ]:
def genRemoveList(OgList, strPrtList):
    newRemoveList = []
    
    for i in OgList:
        for string in strPrtList:
            if string in i:
                print(i)
                newRemoveList.append(i)
    return newRemoveList

In [ ]:
def sqlClause(termList, fieldName):
    owner_string = ""
    for owner in termList:
        if termList.index(owner) != len(termList) -1:
            owner_string = owner_string + f"{fieldName} = '{owner}' Or "
        elif termList.index(owner) == len(termList) -1:
            owner_string = owner_string + f"{fieldName} = '{owner}'"
    return owner_string

In [ ]:
def filterOwners(ownerList):
    for owner in ownerList:
        removeStatus = input(f'Remove {owner} from list? (y/n)')
        if removeStatus == 'y':
            ownerList.remove(owner)
    return ownerList

In [ ]:
def narrowList(ogList):
    ogList = [i for i in ogList if not ('ESTATE' in i or 'SUSAN' in i or 'LLC' in i or 
                                                'COMPANY' in i or 'ASSOC' in i or 'ASSN' in i or
                                        'INC' in i or 'BANK' in i or 'CHURCH' in i or 'SOCIETY' in i or
                                               'RAILWAY' in i or 'PRESBYTERIAN' in i or 'CORP' in i or
                                       'FUND' in i)]
    
    removeList = [i for i in ogList if ('US' in i and not i.startswith('US'))]
    
    ogList = removeTerms(ogList, removeList)
    
    return ogList

In [ ]:
def addAndFillField(fc, termList, newFcName, ownerFieldName):
    
    
    out_featureClass = os.path.join(ws, newFcName)
    arcpy.CopyFeatures_management(fc, out_featureClass)
    arcpy.AddField_management(out_featureClass, 'PublicLand', 'TEXT', 
                              field_length = 1)

    with arcpy.da.UpdateCursor(out_featureClass, 'PublicLand', sqlClause(termList, ownerFieldName)) as cursor:
        for row in cursor:
            row[0] = 'T'
            cursor.updateRow(row)
    

In [ ]:
def addFalseValues(fc):
    with arcpy.da.UpdateCursor(fc, 'PublicLand') as cursor:
        for row in cursor:
            if row[0] != 'T':
                row[0] = 'F'
            cursor.updateRow(row)

In [ ]:
%store -r pubAddList
%store -r refRemoveList

In [ ]:
pubAddList

# Skamania


In [ ]:
myValues = unique_values(r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\SkamaniaCo\Parcel\Parcel\Parcel.gdb\Parcel', 'OwnerLast')

In [ ]:
skaLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\SkamaniaCo\Parcel\Parcel\Parcel.gdb\Parcel'

In [ ]:
searchTerms = ['USA', 'UNITED', 'STATE', 'OREGON', 'WASH', 'COUNTY', 'SKAMANIA', 'CITY', 'GOVERNMENT', 'SCHOOL', 'PUBLIC']
skaPubLand = pubFind(searchTerms, myValues)

In [ ]:
skaPubLand = list(set(skaPubLand))


In [ ]:
removeList = ['BLUE SKY ESTATES HOMEOWNERS ASSOCIATION', 'ESTATE OF WALTER RAYMOND LONG', 
              'FRN WASHINGTON, LLC', 'GIRL SCOUTS OF OREGON &', 'LJ GRAF SKAMANIA COUNTY TIMBER, LLC',
             'MERCY PROPERTIES WASHINGTON I', 'OREGON & WASHINGTON RR NAV CO', 'OREGON CONFERENCE ADVENTIST CHURCHES',
             'OREGON & WASHINTON TELEPHONE', 'PACIFICORP, AN OREGON CORPORATION', 'PORT OF SKAMANIA COUNTY', 'RIVERSIDE ESTATES ASSOCIATION',
             'SENIOR SERVICES OF SKAMANIA', 'SKAMANIA CO HISTORICAL SOCIETY', 'SKAMANIA COMMUNITY CHURCH',
             'SKAMANIA COUNTY CEMETERY', 'SKAMANIA COUNTY COUNCIL ON', 'SKAMANIA COUNTY HOSPITAL DISTRICT',
             'SKAMANIA COUNTY ROD & GUN CLUB', 'SKAMANIA GENERAL STORE LLC', 'SKAMANIA LANDING LLC',
             'SKAMANIA LANDING OWNERS ASSOC', 'SKAMANIA PHYSICAL THERAPY', 'SWIFT CREEK ESTATES',
             'SYNESIS SOFTWARE (USA) INC', 'THE RESORT AT SKAMANIA COVES', 'UNITED TELEPHONE OF THE NW',
             'WASHOUGAL TIMBER TRAILS ASSOC'] + refRemoveList

In [ ]:
skaPubLand = removeTerms(skaPubLand, removeList)

In [ ]:
skaPubLand = skaPubLand + pubAddList

In [ ]:
owner_string = ""
for owner in skaPubLand:
    if skaPubLand.index(owner) != 27:
        owner_string = owner_string + f"OwnerLast = '{owner}' Or "
        
    elif skaPubLand.index(owner) == 27:
        owner_string = owner_string + f"OwnerLast = '{owner}'"
print(owner_string)
        

In [ ]:
skaPubOwners = skaPubLand

In [ ]:
#out_featureClass = os.path.join(ws, 'skaParcels')
#arcpy.CopyFeatures_management(skaLyr, out_featureClass)
#arcpy.AddField_management(out_featureClass, 'PublicLand', 'TEXT', 
 #                         field_length = 1)

#with arcpy.da.UpdateCursor(out_featureClass, 'PublicLand', sqlClause(skaPubLand, 'OwnerLast')) as cursor:
    #for row in cursor:
        #row[0] = 'T'
        #cursor.updateRow(row)

In [ ]:

#arcpy.CopyFeatures_management('public_parcels3', r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb\skamaniaPublicParcels' )

# Wahkiakum

In [ ]:
waLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\WahkiakumCo\Parcels\Parcels.shp'

searchTerms = ['USA', 'UNITED', 'STATE', 'ORE', 'WASH', 
            'COUNTY', 'WAHKIAKUM', 'CITY', 'GOVERNMENT', 'SCHOOL', 'PUBLIC']

myValues = unique_values(waLyr, 'Property_O')

waPublicOwners = pubFind(searchTerms, myValues)

print(waPublicOwners)


In [ ]:
waPubOwners = ['SCHOOL DISTRICT #200','STATE OF WA', 'STATE OF WA DEPT OF FISH & WILDLIFE', 'STATE OF WA DEPT OF GAME', 
             'STATE OF WA DEPT OF HWY', 'STATE OF WA DEPT OF TRANS', 'STATE OF WA DEPT OF WILDLIFE', 'STATE OF WA DNR', 
             'STATE OF WA DNR NATURAL AREAS PROGRAM', 'STATE OF WA FISH & WILDLIFE', 'STATE OF WA HWY #12', 
             'USA BUREAU OF INDIAN AFFAIRS', 'USA FISH & WILDLIFE', 'WAHKIAKUM COUNTY']

In [ ]:
waPubOwners.index('WAHKIAKUM COUNTY')

In [ ]:
sql = sqlClause(waPubOwners, 'Property_O')

In [ ]:
waPubOwners = waPubOwners + pubAddList

In [ ]:
#addAndFillField(waLyr, waPubOwners, 'wahkiakumParcels', 'Property_O')

# Clatsop

In [ ]:
clatsLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ClatsopCo\Clatsop County Secure File Transfer-mXZcBwbyFGajC3KD\Clatsop County Secure File Transfer-mXZcBwbyFGajC3KD\LCEP\LCEP.shp'

searchTerms = ['UNITED', 'STATE', 'OREGON', 'WASH', 
            'COUNTY', 'CLATSOP', 'CITY', 'GOVERNMENT', 'SCHOOL', 'PUBLIC']

myValues = unique_values(clatsLyr, 'OWNER_LINE')

clatsPubOwners = pubFind(searchTerms, myValues)



In [ ]:
clatsPubOwners = [i for i in clatsPubOwners if not ('ESTATE' in i or 'SUSAN' in i)]

In [ ]:
removeList = ['ARCHDIOCESE OF OREGON CITY', 'ARCHDIOCESE OF OREGON CITY', 'ASSEMBLIES OF GOD OREGON DIST', 
              'AST LODGE 2 UNITED FINNISH', 'ASTORIA LODGE #2 UNITED', 'BHRE-OREGON LLC', 'BANK OF ASTORIA AN OREGON CORP', 
              'BELLINGER OREGON QTIP GARY D', 'BIO-OREGON PROPERTIES LLC', 'BUGATTO OREGON PROPERTIES', 
              'BUGATTO OREGON PROPERTIES INC', 'CITY CENTER HOSPITALITY LLC', 'CITY LUMBER COMPANY',
              'CLATSOP CARE & REHAB CENTER DIST', 'CLATSOP CO HISTORICAL SOCIETY', 
              'CLATSOP CO WOMENS RESOURCE INC', 'CLATSOP COMMUNITY ACTION', 'CLATSOP COMMUNITY BANK', 'CLATSOP COMMUNITY COLLEGE',
              'CLATSOP COUNTY DEV TRAIN CTR', 
              'CLATSOP COUNTY DEV TRAIN CTR', 'CLATSOP COUNTY DEV TRAINING CENTER', 'CLATSOP COUNTY DEV TRAINING CENTER', 
              'CLATSOP COUNTY HISTORICAL SOC', 'CLATSOP COUNTY HISTORICAL SOC', 'CLATSOP COUNTY HISTORICAL SOCIETY', 
              'CLATSOP COUNTY HISTORICAL SOCIETY','CLATSOP DEVELOPMENT CO', 'CLATSOP INDUSTRIAL PARK LLC', 'CLATSOP QUALITY CONSTRUCTION', 'CLATSOP RCA LLC', 
              'CLATSOP RENTALS LLC', 'CLATSOP RETIREMENT VILLAGE PROP', 'COVERT KRISTIN WASHBURN/MICHAEL', 
              'EAST OREGONIAN PUBLISHING CO', 'FIRST INTERSTATE BANK NA TR', 'FIRST INTERSTATE BANK OF OR TR', 
              'FIRST NATIONAL BANK OF OREGON', 'FIRST REPUBLIC TRUST COMPANY', 'FIRST UNITED METHODIST CHURCH', 
              'FORT CLATSOP HISTORICAL ASSOC', 'GOVERNMENT NATIONAL MORTGAGE ASSOCIATION', 'GREATER OREGON BEHAVIORAL HEALTH', 
              'GREATER OREGON BETTER HOUSING INITIATIVE', 'GREEN FELICITY HOPE', 'HAMLET HISTORICAL SCHOOLHOUSE', 
              'INTERSTATE AMUSEMENT CO', 'KUCOBICARE OREGON LLC', 'LEWIS & CLARK OREGON TIMBER LLC', 
              'NORTHWEST OREGON HOUSING ASSOC INC', 'NORTHWEST OREGON HOUSING AUTH', 'NORTHWEST OREGON HOUSING AUTHORITY', 
              'OREGON BANK THE', 'OREGON BANK THE TRUSTEE', 'OREGON CONFERENCE ADVENTIST CHURCHES', 'OREGON DIST COUNCIL ASSEMBLY',
              'OREGON MILITARY DEPARTMENT', 'OREGON PAC DIST CHURCH OF THE NAZARENE',
              'OREGON PROPERTY INVESTMENTS GROUP LLC', 'OREGON STATE UNIVERSITY FOUNDATIONI','OREGON STATE UNIVERSITY FOUNDATIONI', 
              'PATTON OREGON PROPERTIES LLC', 'PROVIDENCE HEALTH SYSTEM - OREGON', 'PROVIDENCE HEALTH SYSTEM OREGON',
              'PUBLIC BREWING COMPANY', 'RECOLOGY WESTERN OREGON INC', 'RIVER CITY INVESTMENT CO LLC', 'RIVER CITY INVESTMENT COMPANY', 
              'RIVER CITY INVESTMENT COMPANY LLC', 'RIVERCITY INVESTMENT CO LLC', 'S & S AKHAVAN SARAF OREGON PROP',
              'SEASIDE CITY OF URBAN RENEWAL AGENCY', 'SHORT TERM BIBLE SCHOOL', 'SHORT TERM BIBLE SCHOOLS', 'SHORT TERM BIBLE SCHOOLS INC', 
              'SOUTH COUNTY CMTY FOOD BANK INC','TIMES THEATRE & PUBLIC HOUSE', 'TRI-CITY PROPERTIES LLC', 'TWO C S PROPERTIES OF OREGON LLC', 
              "TWO C'S PROPERTIES OF OREGON LLC", 'U S NATIONAL BANK OF OREGON', 'UNION HEALTH DISTRICT OF CLATSOP COUNTY', 
              'UNION HEALTH DISTRICT OF CLATSOP COUNTY', 'UNITED METHODIST CHURCH OF SEASIDE', "UNITED PAPERWORKERS INTERNAT'L", 
              'UNITED STATES BAKERY', 'UNITED STATES BAKERY','UNITED STATES OF AMERICA BPA', 'UNITED STATES OF AMERICA BPA', 
              'WARRENTON CITY/MUNICIPAL CORP', 'WASHBURN CHRISTINE C', 'WASHINGTON ALLYN J', 'WASHINGTON AMANDA', 'WASHINGTON ELIZABETH J', 
              'WASHINGTON GLEN LEIGHTON', 'WASHINGTON TAMMY L', 'WEST OREGON ELECTRIC COOP', 'WESTERN OREGON WASTE', 
              'WESTERN OREGON WASTE COAST INC', 'WESTERN STATE BUILDERS INC'] + refRemoveList
              
              
              
              

In [ ]:
clatsPubOwners = removeTerms(clatsPubOwners, removeList)

In [ ]:
clatsPubOwners = clatsPubOwners + pubAddList

In [ ]:
#addAndFillField(clatsLyr, clatsPubOwners, 'clatsopParcels', 'OWNER_LINE')

# Clark

#### Anyone who needs to obtain the property owner name data must submit an email request to the Clark County Treasurer's Office (treasoff@clark.wa.gov) which specifies that they are requesting access to the property owner name data that are maintained by Clark County GIS.


In [ ]:
clarkLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ClarkCo\Ownership\PublicLandOwnership.shp'


myValues = unique_values(clarkLyr, 'PACSOwner')
clarkPubOwners = myValues



# Cowlitz

In [ ]:
cowLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\CowlitzCo\CoParcels\Parcels_Oct_2020\Parcels_Oct_2020.shp'
myValues = unique_values(cowLyr, 'DEED_HOLDE')
searchTerms = ['UNITED', 'STATE', 'US ', 'USA ', 'OREGON', 'WASH', 
            'COUNTY', 'COWLITZ', 'CITY', 'GOVERNMENT', 'SCHOOL', 'PUBLIC']
cowPubOwners = pubFind(searchTerms, myValues)


In [ ]:
cowPubOwners = [i for i in cowPubOwners if not ('ESTATE' in i or 'SUSAN' in i)]
removeUs = [i for i in cowPubOwners if ('US' in i and not i.startswith('US'))]

In [ ]:
cowPubOwners = removeTerms(cowPubOwners, removeUs)

In [ ]:
removeList = ['47 COWLITZ LLC  (C)', '946 WASHINGTON WAY LLC', 'ASH COUNTY PROPERTIES LLC', 'BYMB WASHINGTON PROPERTIES LLC',
             'CITY VIEW TER II HOMEOWN ASSOC', 'CITY VIEW TERRACE HOMEOWNERS', 'CITY VIEW TERRACES HOMEOWNERS', 
              'CITY VIEW TERRACES II HOMEOWNERS', 'COLUMBIA STATE BANK', 'COLUMBIA/COWLITZ RAILWAY', 'COUNTYWIDE PROPERTIES LLC',
             'COWLITZ COUNTY CASA', 'COWLITZ COUNTY CASA', 'COWLITZ COUNTY HABITAT FOR HUMANITY', 
              'COWLITZ COUNTY HABITAT FOR HUMANITY', 'COWLITZ ENTERPRISES', 'COWLITZ FAMILY HEALTH CENTER',
              'COWLITZ HUMANE SOCIETY', 'COWLITZ INDIAN TRIBE', 'COWLITZ INVESTMENTS LLC', 'COWLITZ LODGE #66 IOOF', 
              'COWLITZ ORCHARDS LLC', 'COWLITZ RIVER MHP LLC', 'COWLITZ TERRACE DEVELOPMENT LTD', 
              'COWLITZ VALLEY WILDLIFE', 'COWLITZ VALLEY WILDLIFE LEAGUE', 'COWLITZ VIEW MEM GDN ENDOW CARE FUND',
              'COWLITZ VIEW MEMORIAL GARDENS', 'COWLITZ WAY PLAZA LLC', 'DMC PROPERTIES WASHINGTON LLC', 
              'FAST TRACK NORTHWEST-WASHINGTON', 'FIRST ASSEMBLY OF GOD COWLITZ COUNTY', 'FIRST ASSEMBLY OF GOD COWLITZ COUNTY', 
              'HARRIS LUMBER CO OREGON CORP', 'JDS DEVELOPMENT-WASHINGTON LLC', 'JONES WASHINGTON STEVEDORING CO',
             'KALAMA UNITED METHODIST CHURCH', 'KEATLEY COWLITZ FARM LLC', 'KELSO CAR WASH LLC', 'KELSO FIRST UNITED METHODIST',
              'LG PROPERTIES OF COWLITZ CO LLC', 'LG PROPERTIES OF COWLITZ COUNTY LLC', 'LG PROPERTIES OF COWLITZ COUNTY LLC', 
              'LGI HOMES - WASHINGTON LLC', 'LONGVIEW CHRISTIAN SCHOOL', 'LONGVIEW CHRISTIAN SCHOOL INC'
            'LONGVIEW UNITED METHODIST CHURCH', 'MAKE IT FOUNDATION AN OREGON NONPROFIT CORPORATION', 
              'OLD REPUBLIC NATIONAL TITLE INSURANCE CO', 'OREGON CONFERENCE ADVENTIST CHURCHES', 
              'OREGON CONFERENCE EDUCATION ASSOCIATION', 'OTTIS FELICITY F', 'PACIFIC WEST TIMBER CO (WASHINGTON) LLC', 
              'PARSON WASHINGTON PROPERTIES LLC', 'ROSE CITY INVESTMENT PROPERTIES LLC', 'SCHOOLCRAFT BRANDEN/KATHERINE', 
              'SCHOOLING GERRI G/WILLIAM D', 'THREE RIVERS CHRISTIAN SCHOOL', 'TWIN CITY BAPTIST CHURCH', 
              'TWIN CITY INVESTMENTS LLC', 'UNITED INVESTMENT MANAGAMENT LLC', 'UNITED PENTECOSTAL CHURCH', 
              'UNITED STATES BAKERY', 'UNITED STATES BAKERY', 'US BANK  L/O', 'US BANK NATIONAL ASSOCIATION N D',
             'US NATURAL RESOURCES INC', 'US WEST COMMUNICATIONS / PACIFIC', 'US WEST COMMUNICATIONS INC',
             'WASH BAPTIST CONV', 'WASH INTERSTATE BROADCASTING INC', 'WASH INTERSTATE BROADCASTING INC', 'WASH MUTUAL BK  LO', 
              'WASHAM JENNIFER E ETAL', 'WASHBURN CAROL A', 'WASHBURN JEFF P/STRANGE TERESA L', 'WASHBURN JEFFREY P/TERESA L', 
              'WASHBURN JORDEN Z/COUTNEY A', 'WASHBURN TERESA L', 'WASHINGTON ALBERT B ETUX', 'WASHINGTON BAPTIST CHURCH CONV', 
              'WASHINGTON BAPTIST CONVENTION', 'WASHINGTON DELLA J  RO', 'WASHINGTON MIRON', 'WASHINGTON MUTUAL BANK  LO', 
              'WASHINGTON PETROLEUM GROUP INC', 'WASHINGTON SQUARE LLC', 'WASHINGTON VICKI M', 'WASHMAN LLC', 
              'WEBSTER MICHAEL R/ANACITY A'] + refRemoveList

In [ ]:
cowPubOwners = removeTerms(cowPubOwners, removeList)

In [ ]:
cowPubOwners = cowPubOwners + pubAddList

In [ ]:
#addAndFillField(cowLyr, cowPubOwners, 'cowlitzParcels', 'DEED_HOLDE')

# Pacific

In [ ]:
pacLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\PacificCo\Tax_Lots.shp'
myValues = unique_values(pacLyr, 'CURRENT_O')
searchTerms = ['UNITED', 'STATE', 'US ', 'USA ', 'OREGON', 'WASH', 
            'COUNTY', 'PACIFIC', 'CITY', 'GOVERNMENT', 'SCHOOL', 'PUBLIC']
pacPubOwners = pubFind(searchTerms, myValues)


In [ ]:
pacPubOwners = [i for i in pacPubOwners if not ('ESTATE' in i or 'SUSAN' in i or 'LLC' in i or 
                                                'COMPANY' in i or 'ASSOC' in i or 'INC' in i or 
                                               'BANK' in i or 'CHURCH' in i or 'SOCIETY' in i or
                                               'RAILWAY' in i or 'PRESBYTERIAN' in i)]
removeUs = [i for i in pacPubOwners if ('US' in i and not i.startswith('US'))]
pacPubOwners = removeTerms(pacPubOwners, removeUs)

In [ ]:
removeList = ['ANNIE WRIGHT SCHOOL,', 'DURYEE & SAUNDERS PACIFIC COUNTY REAL*', 'DURYEE & SAUNDERS PACIFIC COUNTY REAL*', 
              'EAST OREGONIAN PUBLISHING CO,', 'FIA TIMBER GROWTH & VALUE WASHINGTON *', 'FRIENDS OF CHINOOK SCHOOL,', 
              'LEWIS COUNTY COMMUNITY HEALTH SERVICE*', 'PACIFIC COAST CRANBERRY RESEARCH,', 'PACIFIC COUNTY ANGLERS,', 
              'PACIFIC COUNTY ANGLERS,', 'PACIFIC EYE CLINIC,', 'PACIFIC NW ANNUAL CONF/UNITED METHODI*', 
              'PACIFIC NW ANNUAL CONF/UNITED METHODI*', 'PACIFIC PLACE APT LMT,',"RAYMOND WASH CONG OF JEHOVAH'S WITNES*", 
             'SCHOOLING, JACK E & DONELLE J', 'STATEN, JACQUALINE M', 'SUNRISE WASHINGTON SUBSIDIARY CO,',
             'WASHAM, DANIEL & HEIDI & GLENN', 'WASHAM, GLENN', 'WASHINGTON CONFERENCE OF SEVENTH DAY *', 
              'WESTERN WASH CONS BAPTIST CAMPING ASS*', 'WESTERN WASH CONSERV BAPTIST,', 'WESTERN WASH CORP 7TH DAY ADVEN,'] + refRemoveList

In [ ]:
pacPubOwners = removeTerms(pacPubOwners, removeList)

In [ ]:
pacPubOwners = pacPubOwners + pubAddList

In [ ]:
#addAndFillField(pacLyr, pacPubOwners, 'pacificParcels', 'CURRENT_O')

# Multnomah

In [ ]:
multLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\MultCo\Taxlot_Parcels.shp'
myValues = unique_values(multLyr, 'OWNER1')
searchTerms = ['UNITED', 'STATE', 'US ', 'USA ', 'OREGON', 'WASH', 
            'COUNTY', 'MULT', 'CITY', 'GOVERNMENT', 'SCHOOL', 'PUBLIC', 'METRO']
multPubOwners = pubFind(searchTerms, myValues)

In [ ]:
multPubOwners = narrowList(multPubOwners)

In [ ]:
removeList1 = ['AAA OREGON/IDAHO', 'ADULT LEARNING SYSTEMS OF OREGON', 'APOSTOLIC FAITH MISSION OF PORTLAND OREGON', 
              'ARCHDIOCESE OF PORTLAND IN OREGON', 'ASIAN BUDDHIST COMMUNITY CONGREGATION OF OREGON', 
              'ASSEMBLIES OF GOD DISTRICT OF OREGON', 'AUTOMOBILE CLUB OF OREGON', 'BIAVA JAIME E & WASHBURN KELLY N', 
              'BROADWAY & WASHINGTON CONDO OWNERS ASSN', 'BROWN RICHARD P & WASHINGTON LAURA L', 'CASE PATRICK R & CASE FELICITY M', 
              'CCC-INTERSTATE APARTMENTS LP', 'CENTRAL CATHOLIC HIGH SCHOOL PORTLAND OREGON', 'CENTRAL CATHOLIC HIGH SCHOOL PORTLAND OREGON', 
              'CENTRAL CITY CONCERN', 'CHABAD-LUBAVITCH OF OREGON', 'CHRISTIE STATER ANDERSEN NONEXEMPT SHARE TR', 'CITY TEAM MINISTRIES', 
              'CITY VIEW VENTURES LP', "CITYBIKES WORKERS' COOPERATIVE", 'CLATSOP COUNTY DEVEL TRAINING CENTER ASSN', 
              'COLUMBIA CHRISTIAN SCHOOLS', 'CONFEDERATED TRIBES OF SILETZ INDIANS OF OREGON', 
              'CONFEDERATED TRIBES OF THE GRAND RONDE COMMUNITY OF OREGON', 'CONNIE K SCHOOLEY TR', 'DIOCESE OF OREGON', 
              'DONALD A WASHBURN REV TR & CHRISTINE C WASHBURN REV TR', 'FAIRFIELD INTERSTATE LANES LP',
              'FAITHFUL SAVIOR MINISTRIES COMMUNITY SCHOOL', 'FILM ACTION OREGON', 'FRANCISCAN LIFE EDUCATION CENTER OF OREGON',
              "FRENCH AMERICAN INT'L SCHOOL", 'GAMELIN-OREGON ASSN', 'GIRL SCOUTS OF OREGON AND', 'GRACE CITY PORTLAND',
             "HARTLEY SURVIVOR'S TR & HARTLEY STATE FAMILY TR", 'HAWASH MAHER & HAWASH LISA', 'HEYL JONATHAN H & SCHOOLEY SARA E', 
              'HOLLY SALES & SERVICE OF OREGON CO', 'HOLY TRINITY GREEK ORTHODOX CATHEDRAL OF OREGON', 
              'HOME FORWARD(LEASED MULTIPLE TENANTS)', 'IBEW & UNITED WORKERS FEDERAL CREDIT UNION', 'IU MIEN PUBLIC BUDDHA TEMPLE', 
              'IVY SCHOOL-A MONTESSORI CHARTER SCHOOL', 'JONES OREGON STEVEDORING CO', 'JUDITH E GOULD EXEMPTION TR -OREGON ELECTED', 
              'KAISER HEALTH PLAN OF OREGON', 'KALLURI FAMILY OREGON TR', 'METROEAST COMMUNITY MEDIA', 'METROKE MICHAEL J TR & METROKE MARCIA G TR', 
               'METROPOLITAN FAMILY SERVICES', 'METROPOLITAN LIFE INSURANCE CO']

In [ ]:
removeList2 = ['KENTON LODGE NO 145 A F & A M OF OREGON', 'KOWASH NATHANIEL S & KOWASH LESLIE A', 'LUISE L LANE BY-PASS OREGON QTIP TR', 
               'LUTHERAN FAMILY SERVICE OF OREGON AND SW WASHINGTON', 'LUTHERAN FAMILY SERVICE OF OREGON AND SW WASHINGTON', 
               'MEIGS PATRICK & NUNLEY FELICITY','METANOIA PEACE COMMUNITY UNITED', 'MICRO ENTERPRISE SERVICES OF OREGON', 
               'MULTNOMAH AMATEUR ATHLETIC CLUB', 'MULTNOMAH ATHLETIC CLUB', 'MULTNOMAH LEARNING ACADEMY', 
               'MULTNOMAH MONTHLY MEETING OF FRIENDS', 'MULTNOMAH UNIVERSITY', 'NEIGHBORHOOD UNITED PENTECOSTAL CH OF PORT ORE', 
               'NEW DAY ANANDA MARGA SCHOOL OF PORTLAND', 'NYE CHRISTOPHER M & MULTZ POPPI', 'OCF JOSEPH E WESTON PUBLIC FOUNDATION', 
               'OREGON AFL-CIO', 'OREGON AFSCME COUNCIL 75', 'OREGON BAPTIST CONVENTION', 'OREGON BAPTIST RETIREMENT HOME', 
               'OREGON BAPTIST RETIREMENT HOMES', 'OREGON BAPTIST STATE CONVENTION PLEASANT VALLEY COMM BAP', 
               'OREGON BAPTIST STATE CONVENTION PLEASANT VALLEY COMM BAP', 'OREGON BECKNELL INVESTORS', 
               'OREGON BEVERAGE RECYCLING COOPERATIVE', 'OREGON BUDDHIST TEMPLE', 'OREGON CATHOLIC PRESS', 
               'OREGON CITY BUILDING LP', 'OREGON CITY BUILDING LP', 'OREGON CITY FURNITURE CO', 'OREGON CITY FURNITURE CO', 
               'OREGON COMMUNITY FOUNDATION', 'OREGON CONSISTORY NO 1 OF PORTLAND', 'OREGON HEALTH & SCIENCE UNIVERSITY', 
               'OREGON ISLAMIC CHAPLAIN ORG', 'OREGON LEATHER CO', 'OREGON LIMITED', 'OREGON MEDITATION CENTER', 
               'OREGON PACIFIC INVEST & DEVEL CO', 'OREGON PROPERTY FACTORS', 'OREGON PUBLIC BROADCASTING', 
               'OREGON PUBLIC BROADCASTING', 'OREGON PUBLIC BROADCASTING FOUNDATION', 'OREGON PUBLIC BROADCASTING FOUNDATION',
              'OREGON STUDENT FOUNDATION', 'OREGON TRAIL POST #150 AMERICAN LEGION', 'OREGON TRANSFER CO', 
               'OREGON WASH RR & NAVIGATION CO', 'OREGON WASH RR & NAVIGATION CO', 'OREGON WORSTED CO ETAL', 
               'OREGON YACHT CLUB LTD', 'OREGONIANS FEDERAL CREDIT UNION', 'OSTROM ZACHARY & WASHATKO NOAH', 'OUR UNITED VILLAGE', 
               'OUR UNITED VILLAGES', 'OWENS STEVEN & WASHINGTON BECKY', 'PATERSON OREGON PROPERTY TR', 
               'PIENKOS ANDREW P & STATES KATHLEEN W', "PORTLAND STATE UNIVERSITY - LEASED CAF'E YUMM", 'PROTESTANT EPISCOPAL BISHOP OF OREGON', 'PROVIDENCE HEALTH & SERVICES-OREGON',
              'ROBINSON STATES W & KUNI KRISTINE', 'ROMAN CATHOLIC ARCHBISHOP OF PORTLAND IN OREGON', 'ROSE CITY CEMETERY', 
               "ROSE CITY CEMETERY ASS'N", 'ROSE CITY HOLDINGS', 'ROSE CITY INVESTMENT CO', 'SCHADE TIMOTHY L & WASHBURN ALLISYN E',
              'SCHOOLER JASON & SCHOOLER CYMBRIE', 'SCHOOLEY CHERI L & SCHOOLEY JOHN', 'SCHOOLEY TODD E & BACHELDER NATHANIEL A', 
               'SCHOOLMASTER SALLY', 'ST MARY CATHOLIC CATHEDRAL OF IMMACULATE CONCEPTION OF PORTLAND OREGON', 
               "ST STEPHEN'S EPISCOPAL PARISH PORTLAND OREGON", 'STATE ANTHONY E & STATE PAMELA K', 'STATE THOMAS E & STATE KATHERINE M', 
               'STATEN HARRY & STATEN AMY', 'STATEN JUDITH M & STATEN KENNETH W', 'STATEN-ALBRECHT KATHLEEN C', 'STATES DEREK', 
               'STEWART RICHARD & STEWART FELICITY A', 'THE CONFEDERATED TRIBES OF THE GRAND RONDE COMMUNITY OF OREGON', 
               'THE FRENCH AMERICAN INTERNATIONAL SCHOOL', 'THE INTERNATIONAL SCHOOL', 'THE JAPANESE GARDEN FOUNDATION OF OREGON', 
               'THE OREGON COMMUNITY FOUNDATION', 'TRAILS CLUB OF OREGON', 'TRINITY EPISCOPAL CATHEDRAL PORTLAND OREGON',
              'TUCKER MAXON SCHOOL', 'UNITED BROTHERHOOD OF CARPENTERS & JOINERS OF AMERICA', 'UNITED FINANCE CO', 'UNITED REALTY CO', 
               'UNITED STATES BAKERY', 'UNITED STATES BAKERY', 'UNITED STATES MISSION', 'UNITED STATES MISSION',
              'UNITED TELEPHONE CO NW', 'UNITED WAY OF THE COLUMBIA-WILLAMETTE', 'UNIVERSITY OF WESTERN STATES',
              "US NAT'L BK OF OR", "US NAT'L BK OF OR TR", 'US SPRINT COMMUNICATIONS CO LTD PARTNERSHIP', 'VAN-LANG VIETNAMESE NONPROFIT SCHOOL',
               'WASH BARBARA A & WASH JOEY D', 'WASH DAVID L & DURKEE JOANNE L', 'WASH DONNA O', 'WASH JOEY D', 'WASH MAN L L C', 
               'WASHAM DENNIS W & WASHAM PAMELA J', 'WASHBURN BILLY J & WASHBURN KATHLEEN G', 'WASHBURN BRADLEY & BENSON KRISTIN', 
               'WASHBURN BRADLEY & BENSON KRISTIN R', 'WASHBURN CAROL S TR & WASHBURN JOE D TR', 'WASHBURN CHARLES J & WASHBURN JENNIFER L', 
               'WASHBURN DEANNE L TR', 'WASHBURN DIANA L & WASHBURN ROBERT B', 'WASHBURN DONALD & WASHBURN LISA A L', 'WASHBURN DONALD A', 
               'WASHBURN GARY A & WASHBURN DIANE P', 'WASHBURN HEATHER', 'WASHBURN HOLLY B', 'WASHBURN MARY A', 
               'WASHBURN MISTY A & WASHBURN DENNIS W JR', 'WASHBURN ROBERT A & WASHBURN WILMA J', 'WASHBURN SHELBY & WASHBURN SETH', 
               'WASHBURN SHELLEY K', 'WASHBURN THOMAS M & WASHBURN KELLI', 'WASHBURN THOMAS W', 'WASHBURN VA LERA LM', 
               'WASHBURN WARREN R & ANDERSON TAMARA L', 'WASHBURN WILLOW A', 'WASHBURN ZAKARY B & WASHBURN BILLIE J', 
               'WASHCHUK MATTHEW & BOYD KRISTEN', 'WASHINGTON AARON', 'WASHINGTON ALICE', 'WASHINGTON ANDRE L & RAE CYNTHIA L', 
               'WASHINGTON CARLTON C & WASHINGTON HELEN L', 'WASHINGTON CHARLES R & TRAN QUAN L', 'WASHINGTON DARYL N & WASHINGTON GLENN R', 
               'WASHINGTON DEBBY L & WASHINGTON ARCHIE', 'WASHINGTON DWAYNE D', 'WASHINGTON EDWARD J', 'WASHINGTON ELIZABETH C', 
               'WASHINGTON ELIZABETH J', 'WASHINGTON ERWIN V & WASHINGTON BARBARA M', 'WASHINGTON FEDERAL', 'WASHINGTON FEDERAL SAVINGS', 
               'WASHINGTON GERALINE', 'WASHINGTON GREGORY L & SWOPES KIMBERLY', 'WASHINGTON JASON E & WASHINGTON MICHELLE L', 
               'WASHINGTON JOSEPH D & WASHINGTON MURIEL O', 'WASHINGTON KEVIN W', 'WASHINGTON KRISANN & WASHINGTON DARNELL', 
               'WASHINGTON LATONIA J', 'WASHINGTON LAURIE K & WASHINGTON JIMMIE', 'WASHINGTON LINDA B', 'WASHINGTON LODGE NO 46 AF & AM', 
               'WASHINGTON MICHAEL R & WASHINGTON PAULETTE D', 'WASHINGTON MOLLY & WASHINGTON RAKEEM A', 'WASHINGTON MYRTLE J', 
               'WASHINGTON OMAR R', 'WASHINGTON PAMELA & ROYER TORRENCE', 'WASHINGTON REBECCA L & WASHINGTON TERRY L', 'WASHINGTON RICK', 
               'WASHINGTON ROBIN A', 'WASHINGTON ROSA L', 'WASHINGTON ROY & WASHINGTON PEGGY', 'WASHINGTON SALLY A', 
               'WASHINGTON SUZANNE-50% & SARAGOZA CONSUELO-50%', 'WASHINGTON SUZANNE-60% & SARAGOZA CONSUELO C-40%', 
               'WASHINGTON TYRONE E & WASHINGTON NEELY E', 'WASHINGTON VERNETTE', 'WASHINGTON WILLIE & WASHINGTON BERTHA F', 
               'WASHINGTON YVETTE A', 'WASHMAN L L C', 'WASHTOK CHRISTOPHER & WASHTOK AMY', 
               "WEST CONGREGATION OF JEHOVAH'S WITNESSES GRESHAM OREGON", 'WEST HILLS CHRISTIAN SCHOOL', 
               'WESTERN UNITED LIFE ASSURANCE CO', 'WILSON TIMOTHY J & SCHOOLER MARY L', 'WOOD VILLAGE CITY OF', 
               'ZEN COMMUNITY OF OREGON']

In [ ]:
removeList = removeList1 + removeList2 + refRemoveList

In [ ]:
multPubOwners = removeTerms(multPubOwners, removeList)

In [ ]:
multPubOwners = list(dict.fromkeys(multPubOwners))

In [ ]:
multPubOwners = multPubOwners + pubAddList

In [ ]:
#addAndFillField(multLyr, multPubOwners, 'multnomahParcels', 'OWNER1')

# Columbia

In [ ]:
colLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb\columbiaDoNotDelete'

searchTerms = ['UNITED', 'STATE', 'OREGON', 'WASH', 
            'COUNTY', 'COLUMBIA', 'CITY', 'GOVERNMENT', 'SCHOOL', 'PUBLIC']

myValues = unique_values(colLyr, 'OWNER')

colPubOwners = pubFind(searchTerms, myValues)




In [ ]:
colPubOwners = narrowList(colPubOwners)

In [ ]:
colRemoveList = ['ARCHDIOCESE OF PORTLAND IN OREGON', 'COLUMBIA BUILDING CO', 'COLUMBIA CO DRG VECTOR CONTROL', 
                 'COLUMBIA COMMUNITY MENTAL HEALTH', 'COLUMBIA COUNTY HABITAT FOR HUMANITY', 'COLUMBIA COUNTY HABITAT FOR HUMANITY',
                'COLUMBIA COUNTY NATAL GRANGE', 'COLUMBIA COUNTY NATAL GRANGE', 'COLUMBIA COUNTY POMONA GRANGE', 
                 'COLUMBIA COUNTY POMONA GRANGE', 'COLUMBIA COUNTY SHRINE CLUB', 'COLUMBIA COUNTY SHRINE CLUB',
                 'COLUMBIA HILLS DEVELOP CO', 'COLUMBIA HILLS RETIREMENT CTR', 'COLUMBIA PARK LIMITED PARTNERSHIP', 
                 'COLUMBIA PHOENIX DEVELOPMENT', 'EAST RIVER FELLOWSHIP @ RAINER OREGON IN', 'HI-SCHOOL PHARMACY', 
                 'LOWER COL SUNDAY SCHOOL FELLOWSHIP', 'OREGON & NATIONAL SOC OF DAUGHTERS OF AM', 'SAFE OF COLUMBIA COUNTY', 
                 'SAFE OF COLUMBIA COUNTY', 'UNITED STATES GYPSUM', 'UNITED STATES GYPSUM', 'WASHAM JENNIFER E', 'WASHBURN CHARLENE', 'WASHBURN CLINTON C & MARCIA J', 
                 'WASHBURN STEPHEN C AND REBECCA J', 'WASHBURN WILLIAM W JR & DIANA L', 'WASHINGTON DUANE & DEBRA K', 'WASHINGTON KHRIS', 
                 'WEST OREGON ELECTRIC COOP', 'ZEN COMMUNITY OF OREGON'] + refRemoveList

In [ ]:
colPubOwners = removeTerms(colPubOwners, colRemoveList)

In [ ]:
colPubOwners = list(dict.fromkeys(colPubOwners))

In [ ]:
colPubOwners = colPubOwners + pubAddList

In [ ]:
#addAndFillField(colLyr, colPubOwners, 'colParcels', 'OWNER')

# List to Export

publicOwners = {'mult': multPubOwners, 'ska': skaPubOwners, 'wa': waPubOwners, 'cla': claPubOwners, 'cow': cowPubOwners, 
                'pac': pacPubOwners}

%store publicOwners

coLyrs = {'mult' : multLyr, 'ska': skaLyr, 'wa': waLyr, 'cla': claLyr, 'cow': cowLyr, 'pac': pacLyr}

# Create MegaList and Filter Out Discretionary Categories


In [ ]:
megaList = colPubOwners + multPubOwners + pacPubOwners + cowPubOwners + clatsPubOwners + waPubOwners + skaPubOwners + clarkPubOwners

In [ ]:
print(megaList)

In [ ]:
discRemove = ['PORT OF', 'SCHOOL', 'EDUC', 'UNIVER', 'COLLEGE', 'LEAS', 'PUD',
             'UTILITY', 'UTILITIES', 'UTITLITIES', 'UTIL DIST','PUBLIC WORKS', 'WATER WORKS', 'WATER BUREAU', 
              'CEMETERY', 'HOSPITAL', 'FIRE AND RESCUE', 'FIRE & RESCUE', 'LIBRAR',
             'DRAIN', 'HIGHER ED', 'HIGHWAY', 'HWY', 'FIRE/RESCUE',
             'FIRE DIST', 'PORT DISTRICT', 'BOYS & GIRLS', 'EVA MARIA DEMETRO',
             'ACADEMY', 'HEALTHCARE', 'FIRE PROTECTION']

In [ ]:
newMegaList, megaRemove = removePart(megaList, discRemove)

In [ ]:
cityRemove = [i for i in newMegaList if ('CITY' in i)]

In [ ]:
cityRemove = [i for i in cityRemove if not ('PARK' in i or 'METRO' in i or 'ENVIRONMENTAL' in i or 'WATER RES' in i or 
                                            'HOUSING AUTHORITY' in i or 'WILDLIFE' in i)]

In [ ]:
pubOwnersList = [colPubOwners, multPubOwners, pacPubOwners, cowPubOwners, clatsPubOwners, waPubOwners, skaPubOwners, clarkPubOwners]

In [ ]:
for ownList in pubOwnersList:
    ownList = removeTerms(ownList, megaRemove)

In [ ]:
for ownList in pubOwnersList:
    ownList = removeTerms(ownList, cityRemove)

## Create layers for all Counties

In [ ]:
out_featureClass = os.path.join(ws, 'skaParcels')
arcpy.CopyFeatures_management(skaLyr, out_featureClass)
arcpy.AddField_management(out_featureClass, 'PublicLand', 'TEXT', 
                         field_length = 1)
with arcpy.da.UpdateCursor(out_featureClass, 'PublicLand', sqlClause(skaPubLand, 'OwnerLast')) as cursor:
    for row in cursor:
        row[0] = 'T'
        cursor.updateRow(row)


In [ ]:
addAndFillField(clarkLyr, clarkPubOwners, 'clarkParcels', 'PACSOwner')
addAndFillField(skaLyr, skaPubOwners, 'skaParcels', 'OwnerLast')
addAndFillField(waLyr, waPubOwners, 'wahkiakumParcels', 'Property_O')
addAndFillField(clatsLyr, clatsPubOwners, 'clatsopParcels', 'OWNER_LINE')
addAndFillField(cowLyr, cowPubOwners, 'cowlitzParcels', 'DEED_HOLDE')
addAndFillField(pacLyr, pacPubOwners, 'pacificParcels', 'CURRENT_O')
addAndFillField(multLyr, multPubOwners, 'multnomahParcels', 'OWNER1')
addAndFillField(colLyr, colPubOwners, 'colParcels', 'OWNER')

In [ ]:
ownerFields = ['OWNER1', 'OwnerLast', 'Property_O', 'OWNER_LINE', 'DEED_HOLDE', 'CURRENT_O', 'OWNER', 'PACSOwner']

In [ ]:
countyLyrs = ['multnomahParcels', 'skaParcels', 'wahkiakumParcels', 
              'clatsopParcels', 'cowlitzParcels', 'pacificParcels', 'colParcels', 'clarkParcels']

In [ ]:
for county in countyLyrs:
    addFalseValues(county)